### Imports


In [1]:
import numpy as np
import pandas as pd

import sys
import os



## Preprossessing

In [2]:
# Import from src/ and config/
from src.preprocessing import preprocess_aachen_dataset
from config.defaults import Config

In [3]:
# Load the default configuration
config = Config()

# Preprocess the dataset for classification (CNN)
preprocessed_classification = preprocess_aachen_dataset(
    data_path=config.data_path,
    test_cell_count=config.test_cell_count,
    random_state=config.random_state,
    log_transform=config.log_transform,
    classification=True  # Explore classification (CNN) output
)

# Preprocess the dataset for regression (LSTM)
preprocessed_regression = preprocess_aachen_dataset(
    data_path=config.data_path,
    test_cell_count=config.test_cell_count,
    random_state=config.random_state,
    log_transform=config.log_transform,
    classification=False  # Explore regression (LSTM) output
)

### Data Load

In [4]:
from src.models import load_preprocessed_data

model_type = "regression"

# Load preprocessed data
X_train, X_val, X_test, y_train, y_val, y_test, metadata = load_preprocessed_data(
    model_type, config.eol_capacity
)

2025-03-03 09:45:38,604 - INFO - Loaded preprocessed data and metadata for regression with EOL 0.65


In [5]:
# Shape of the data
print("X_train shape:", X_train.shape)
print("X_val shape:", X_val.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_val shape:", y_val.shape)
print("y_test shape:", y_test.shape)
metadata


X_train shape: (4513, 120, 1)
X_val shape: (1129, 120, 1)
X_test shape: (391, 120, 1)
y_train shape: (4513,)
y_val shape: (1129,)
y_test shape: (391,)


{'y_max': 995.0,
 'max_sequence_length': 120,
 'eol_capacity': 0.65,
 'classification': False,
 'timestamp': '20250303_094538'}

## Model build

In [6]:
from src.models import build_lstm_model, train_lstm_model, load_saved_model

In [7]:
# check for saved model
saved_model = load_saved_model(model_type, config.eol_capacity, config)

2025-03-03 09:45:42,083 - INFO - No saved model found for regression with EOL 0.65


In [17]:
# Build and train the LSTM model
model = build_lstm_model((metadata["max_sequence_length"], 1), config)

2025-03-03 08:44:49,642 - INFO - LSTM model built for regression with config: Config(data_path='data/raw/Degradation_Prediction_Dataset_ISEA.mat', eol_capacity=0.65, test_cell_count=3, random_state=42, log_transform=False, classification=False, seq_len=120, train_split_ratio=0.8, val_split_ratio=0.2, lstm_units=32, dropout_rate=0.2, dense_units=16, learning_rate=0.001, clipnorm=1.0, patience=15, batch_size=32, epochs=50)


In [9]:

history = train_lstm_model(model, X_train, y_train, X_val, y_val, config)


Epoch 1/50
139/142 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0793 - mae: 0.2023
Epoch 1: val_loss improved from inf to 0.00468, saving model to experiments/models/lstm_regression_eol65_20250303_082852_best.keras
142/142 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - loss: 0.0780 - mae: 0.2001 - val_loss: 0.0047 - val_mae: 0.0527
Epoch 2/50
141/142 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0077 - mae: 0.0649
Epoch 2: val_loss did not improve from 0.00468
142/142 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - loss: 0.0077 - mae: 0.0649 - val_loss: 0.0048 - val_mae: 0.0510
Epoch 3/50
139/142 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0062 - mae: 0.0589
Epoch 3: val_loss improved from 0.00468 to 0.00379, saving model to experiments/models/lstm_regression_eol65_20250303_082852_best.keras
142/142 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - loss: 0.0062 - mae: 0.0589 - val_loss: 0.0038 - val_mae: 0.0487
Epoch 4/50
141/142 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0056 - mae: 0.0555
Epoch 4: val_loss did not improve

2025-03-03 08:30:38,695 - WARNING - The `save_format` argument is deprecated in Keras 3. We recommend removing this argument as it can be inferred from the file path. Received: save_format=keras
2025-03-03 08:30:38,705 - INFO - Final LSTM model saved to experiments/models/lstm_regression_eol65_20250303_083038_final.keras in native Keras format
2025-03-03 08:30:38,705 - INFO - LSTM model trained successfully with config: Config(data_path='data/raw/Degradation_Prediction_Dataset_ISEA.mat', eol_capacity=0.65, test_cell_count=3, random_state=42, log_transform=False, classification=False, seq_len=120, train_split_ratio=0.8, val_split_ratio=0.2, lstm_units=32, dropout_rate=0.2, dense_units=16, learning_rate=0.001, clipnorm=1.0, patience=15, batch_size=32, epochs=50)


In [20]:
from src.evaluation import evaluate_lstm_model

y_max = preprocessed_regression["y_max"]

# Evaluate the model
# Evaluate the model on the test set, reporting progress
test_loss, test_mae = model.evaluate(X_test, y_test, verbose=1)
    
# Generate predictions for the test set silently
y_pred = model.predict(X_test, verbose=0).flatten()

# Rescale predictions and actual values to the original RUL range using y_max
y_pred_rescaled = y_pred * y_max
y_test_rescaled = y_test * y_max

# Calculate the rescaled MAE for better interpretability
mae_rescaled = np.mean(np.abs(y_test_rescaled - y_pred_rescaled))
    
# Print training and evaluation results for verification
print("Training completed successfully!")
print(f"Training Loss: {history['loss'][-1]:.4f}")
print(f"Validation Loss: {history['val_loss'][-1]:.4f}")
print(f"Test Loss: {test_loss:.4f}")
print(f"Test MAE (rescaled): {test_mae:.4f}")

mae_rescaled

13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.3117 - mae: 0.5216 
Training completed successfully!
Training Loss: 0.0032
Validation Loss: 0.0027
Test Loss: 0.2396
Test MAE (rescaled): 0.4491


446.8504226044621